In [1]:
import numpy as np 
import pandas as pd 
import os
import cv2
import matplotlib.pyplot as plt

In [2]:
!unzip /kaggle/input/denoising-dirty-documents/test.zip

Archive:  /kaggle/input/denoising-dirty-documents/test.zip
   creating: test/
  inflating: test/1.png              
 extracting: test/10.png             
  inflating: test/100.png            
 extracting: test/103.png            
  inflating: test/106.png            
 extracting: test/109.png            
  inflating: test/112.png            
 extracting: test/115.png            
 extracting: test/118.png            
  inflating: test/121.png            
  inflating: test/124.png            
  inflating: test/127.png            
 extracting: test/13.png             
  inflating: test/130.png            
 extracting: test/133.png            
  inflating: test/136.png            
 extracting: test/139.png            
 extracting: test/142.png            
 extracting: test/145.png            
  inflating: test/148.png            
 extracting: test/151.png            
  inflating: test/154.png            
 extracting: test/157.png            
 extracting: test/16.png             
 extractin

In [3]:
!unzip /kaggle/input/denoising-dirty-documents/train.zip

Archive:  /kaggle/input/denoising-dirty-documents/train.zip
   creating: train/
  inflating: train/101.png           
  inflating: train/102.png           
  inflating: train/104.png           
  inflating: train/105.png           
 extracting: train/107.png           
 extracting: train/108.png           
 extracting: train/11.png            
  inflating: train/110.png           
 extracting: train/111.png           
  inflating: train/113.png           
  inflating: train/114.png           
 extracting: train/116.png           
  inflating: train/117.png           
 extracting: train/119.png           
 extracting: train/12.png            
 extracting: train/120.png           
  inflating: train/122.png           
 extracting: train/123.png           
  inflating: train/125.png           
  inflating: train/126.png           
  inflating: train/128.png           
  inflating: train/129.png           
  inflating: train/131.png           
 extracting: train/132.png           
 extract

In [4]:
!unzip /kaggle/input/denoising-dirty-documents/train_cleaned.zip

Archive:  /kaggle/input/denoising-dirty-documents/train_cleaned.zip
   creating: train_cleaned/
  inflating: train_cleaned/101.png   
  inflating: train_cleaned/102.png   
  inflating: train_cleaned/104.png   
  inflating: train_cleaned/105.png   
  inflating: train_cleaned/107.png   
  inflating: train_cleaned/108.png   
  inflating: train_cleaned/11.png    
  inflating: train_cleaned/110.png   
  inflating: train_cleaned/111.png   
  inflating: train_cleaned/113.png   
  inflating: train_cleaned/114.png   
  inflating: train_cleaned/116.png   
  inflating: train_cleaned/117.png   
  inflating: train_cleaned/119.png   
  inflating: train_cleaned/12.png    
  inflating: train_cleaned/120.png   
  inflating: train_cleaned/122.png   
  inflating: train_cleaned/123.png   
  inflating: train_cleaned/125.png   
  inflating: train_cleaned/126.png   
  inflating: train_cleaned/128.png   
  inflating: train_cleaned/129.png   
  inflating: train_cleaned/131.png   
  inflating: train_cleaned/132

In [5]:
train_filenames = os.listdir('train')
train_cleaned_filenames = os.listdir('train_cleaned')
test_filenames = os.listdir('test')
print("Train files: {}".format(len(train_filenames)))
print("Test files: {}".format(len(test_filenames)))
print("Cleaned Train files: {}".format(len(train_cleaned_filenames)))

Train files: 144
Test files: 72
Cleaned Train files: 144


In [6]:

train_images = [cv2.imread(os.path.join('train', file), 0) for file in train_filenames if file.endswith('.png')]
cleaned_train_images = [cv2.imread(os.path.join('train_cleaned', file), 0) for file in train_cleaned_filenames if file.endswith('.png')]
test_images = [cv2.imread(os.path.join('test', file), 0) for file in test_filenames if file.endswith('.png')]

train_images = [cv2.resize(img, (540, 260)) for img in train_images]
cleaned_train_images = [cv2.resize(img, (540, 260)) for img in cleaned_train_images]
test_images = [cv2.resize(img, (540, 260)) for img in test_images]

print(len(train_images))
print(len(cleaned_train_images))
print(len(test_images))

144
144
72


In [7]:
train_images = np.stack(train_images, axis=0).astype('float32')/255.
cleaned_train_images = np.stack(cleaned_train_images, axis=0).astype('float32')/255.
test_images = np.stack(test_images, axis=0).astype('float32')/255.

In [8]:
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model

from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, UpSampling2D
from tensorflow.keras.models import Model

def encoder(input_img):
  x = Conv2D(32, (3, 3), activation='relu', padding='same', name='encoder_conv1')(input_img)
  x = MaxPool2D((2, 2), padding='same', name='encoder_pool1')(x)
  x = Conv2D(64, (3, 3), activation='relu', padding='same', name='encoder_conv2')(x)
  x = MaxPool2D((2, 2), padding='same', name='encoder_pool2')(x)
  encoded = Conv2D(128, (3, 3), activation='relu', padding='same', name='encoder_conv3')(x)
  return encoded


def decoder(encoded):
  x = Conv2D(128, (3, 3), activation='relu', padding='same', name='decoder_conv1')(encoded)
  x = UpSampling2D((2, 2), name='decoder_upsample1')(x)
  x = Conv2D(64, (3, 3), activation='relu', padding='same', name='decoder_conv2')(x)
  x = UpSampling2D((2, 2), name='decoder_upsample2')(x)
  x = Conv2D(32, (3, 3), activation='relu', padding='same', name='decoder_conv3')(x)
  decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same', name='decoder_conv4')(x)
  return decoded

In [9]:
input_img = Input(shape=(260, 540, 1))

#create autoencoder model
autoencoder = Model(input_img, decoder(encoder(input_img)))
#complile model
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

autoencoder.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 260, 540, 1)]     0         
_________________________________________________________________
encoder_conv1 (Conv2D)       (None, 260, 540, 32)      320       
_________________________________________________________________
encoder_pool1 (MaxPooling2D) (None, 130, 270, 32)      0         
_________________________________________________________________
encoder_conv2 (Conv2D)       (None, 130, 270, 64)      18496     
_________________________________________________________________
encoder_pool2 (MaxPooling2D) (None, 65, 135, 64)       0         
_________________________________________________________________
encoder_conv3 (Conv2D)       (None, 65, 135, 128)      73856     
_________________________________________________________________
decoder_conv1 (Conv2D)       (None, 65, 135, 128)      147584

In [10]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
earlyStop = EarlyStopping(monitor='loss', patience=10, mode='min')
chkPoint = ModelCheckpoint(filepath='final_model.h5', monitor='loss', mode='min', save_best_only=True)
train_history = autoencoder.fit(np.expand_dims(train_images, 3), np.expand_dims(cleaned_train_images, 3),
                epochs=500,
                batch_size=32,
                shuffle=True,
                callbacks = [chkPoint, chkPoint]
               )

Train on 144 samples
Epoch 1/500
144/144 [==============================] - 7s 50ms/sample - loss: 0.5550
Epoch 2/500
144/144 [==============================] - 1s 10ms/sample - loss: 0.3896
Epoch 3/500
144/144 [==============================] - 1s 10ms/sample - loss: 0.3663
Epoch 4/500
144/144 [==============================] - 1s 10ms/sample - loss: 0.3580
Epoch 5/500
144/144 [==============================] - 1s 10ms/sample - loss: 0.3517
Epoch 6/500
144/144 [==============================] - 1s 10ms/sample - loss: 0.3435
Epoch 7/500
144/144 [==============================] - 1s 10ms/sample - loss: 0.3271
Epoch 8/500
144/144 [==============================] - 1s 10ms/sample - loss: 0.2966
Epoch 9/500
144/144 [==============================] - 1s 10ms/sample - loss: 0.2664
Epoch 10/500
144/144 [==============================] - 1s 10ms/sample - loss: 0.2555
Epoch 11/500
144/144 [==============================] - 1s 10ms/sample - loss: 0.2494
Epoch 12/500
144/144 [====================

144/144 [==============================] - 1s 10ms/sample - loss: 0.1253
Epoch 97/500
144/144 [==============================] - 1s 9ms/sample - loss: 0.1253
Epoch 98/500
144/144 [==============================] - 1s 10ms/sample - loss: 0.1250
Epoch 99/500
144/144 [==============================] - 1s 10ms/sample - loss: 0.1249
Epoch 100/500
144/144 [==============================] - 1s 10ms/sample - loss: 0.1246
Epoch 101/500
144/144 [==============================] - 1s 10ms/sample - loss: 0.1245
Epoch 102/500
144/144 [==============================] - 1s 10ms/sample - loss: 0.1242
Epoch 103/500
144/144 [==============================] - 1s 10ms/sample - loss: 0.1241
Epoch 104/500
144/144 [==============================] - 1s 10ms/sample - loss: 0.1239
Epoch 105/500
144/144 [==============================] - 1s 10ms/sample - loss: 0.1243
Epoch 106/500
144/144 [==============================] - 1s 10ms/sample - loss: 0.1244
Epoch 107/500
144/144 [==============================] - 1s 1

144/144 [==============================] - 1s 10ms/sample - loss: 0.1182
Epoch 191/500
144/144 [==============================] - 1s 10ms/sample - loss: 0.1181
Epoch 192/500
144/144 [==============================] - 1s 10ms/sample - loss: 0.1182
Epoch 193/500
144/144 [==============================] - 1s 10ms/sample - loss: 0.1182
Epoch 194/500
144/144 [==============================] - 1s 10ms/sample - loss: 0.1181
Epoch 195/500
144/144 [==============================] - 1s 10ms/sample - loss: 0.1181
Epoch 196/500
144/144 [==============================] - 1s 10ms/sample - loss: 0.1181
Epoch 197/500
144/144 [==============================] - 1s 10ms/sample - loss: 0.1181
Epoch 198/500
144/144 [==============================] - 1s 10ms/sample - loss: 0.1180
Epoch 199/500
144/144 [==============================] - 1s 10ms/sample - loss: 0.1179
Epoch 200/500
144/144 [==============================] - 1s 10ms/sample - loss: 0.1182
Epoch 201/500
144/144 [==============================] - 

144/144 [==============================] - 1s 10ms/sample - loss: 0.1159
Epoch 285/500
144/144 [==============================] - 1s 10ms/sample - loss: 0.1159
Epoch 286/500
144/144 [==============================] - 1s 10ms/sample - loss: 0.1159
Epoch 287/500
144/144 [==============================] - 1s 10ms/sample - loss: 0.1158
Epoch 288/500
144/144 [==============================] - 1s 10ms/sample - loss: 0.1159
Epoch 289/500
144/144 [==============================] - 1s 10ms/sample - loss: 0.1157
Epoch 290/500
144/144 [==============================] - 1s 10ms/sample - loss: 0.1156
Epoch 291/500
144/144 [==============================] - 1s 10ms/sample - loss: 0.1157
Epoch 292/500
144/144 [==============================] - 1s 10ms/sample - loss: 0.1159
Epoch 293/500
144/144 [==============================] - 1s 10ms/sample - loss: 0.1159
Epoch 294/500
144/144 [==============================] - 1s 10ms/sample - loss: 0.1157
Epoch 295/500
144/144 [==============================] - 

144/144 [==============================] - 1s 10ms/sample - loss: 0.1147
Epoch 379/500
144/144 [==============================] - 1s 10ms/sample - loss: 0.1146
Epoch 380/500
144/144 [==============================] - 1s 10ms/sample - loss: 0.1144
Epoch 381/500
144/144 [==============================] - 1s 10ms/sample - loss: 0.1143
Epoch 382/500
144/144 [==============================] - 1s 10ms/sample - loss: 0.1144
Epoch 383/500
144/144 [==============================] - 1s 10ms/sample - loss: 0.1143
Epoch 384/500
144/144 [==============================] - 1s 9ms/sample - loss: 0.1143
Epoch 385/500
144/144 [==============================] - 1s 10ms/sample - loss: 0.1142
Epoch 386/500
144/144 [==============================] - 1s 10ms/sample - loss: 0.1141
Epoch 387/500
144/144 [==============================] - 1s 10ms/sample - loss: 0.1141
Epoch 388/500
144/144 [==============================] - 1s 10ms/sample - loss: 0.1146
Epoch 389/500
144/144 [==============================] - 1

144/144 [==============================] - 1s 10ms/sample - loss: 0.1137
Epoch 473/500
144/144 [==============================] - 1s 10ms/sample - loss: 0.1135
Epoch 474/500
144/144 [==============================] - 1s 10ms/sample - loss: 0.1135
Epoch 475/500
144/144 [==============================] - 1s 9ms/sample - loss: 0.1134
Epoch 476/500
144/144 [==============================] - 1s 10ms/sample - loss: 0.1133
Epoch 477/500
144/144 [==============================] - 1s 10ms/sample - loss: 0.1133
Epoch 478/500
144/144 [==============================] - 1s 10ms/sample - loss: 0.1134
Epoch 479/500
144/144 [==============================] - 1s 10ms/sample - loss: 0.1134
Epoch 480/500
144/144 [==============================] - 1s 10ms/sample - loss: 0.1134
Epoch 481/500
144/144 [==============================] - 1s 10ms/sample - loss: 0.1133
Epoch 482/500
144/144 [==============================] - 1s 10ms/sample - loss: 0.1133
Epoch 483/500
144/144 [==============================] - 1

In [11]:

autoencoder.load_weights('final_model.h5')

test_image = test_images[51]
test_image = test_image.reshape((1, test_image.shape[0], test_image.shape[1], 1))
prediected_img = autoencoder.predict(test_image)

plt.figure(figsize=(20, 20))
plt.subplot(1, 2, 1)
plt.imshow(test_image[0, :, :, 0], cmap='gray')
plt.axis('off')
plt.subplot(1, 2, 2)
plt.imshow(prediected_img[0, :, :, 0], cmap='gray')
plt.axis('off')
plt.show()

OSError: Unable to open file (unable to open file: name = 'final_weights.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)